### Обучение нейросети

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dataset

#### Импортируем датасет

In [4]:
df = dataset.load_dataset()
df

,Unnamed: 0,website_url,cleaned_website_text,Category
0,0,https://www.booking.com/index.html?aid=1743217,official site good hotel accommodation big sav...,Travel
1,1,https://travelsites.com/expedia/,expedia hotel book sites like use vacation wor...,Travel
2,2,https://travelsites.com/tripadvisor/,tripadvisor hotel book sites like previously d...,Travel
3,3,https://www.momondo.in/?ispredir=true,cheap flights search compare flights momondo f...,Travel
4,4,https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...,bot create free account create free account si...,Travel
...,...,...,...,...
1387,1387,http://www.electroshops.com/,electroshops home theater decor interiors seat...,Business/Corporate
1388,1388,http://www.cleanridge.com/,clean ridge soap company clean ridge soap comp...,Business/Corporate
1389,1389,http://www.creativepetgifts.com/,home page pet crafts exquisitely piece handcut...,Business/Corporate
1390,1390,http://www.htmarket.com/,home theater marketplace home theater seating ...,Business/Corporate


### Обучаем Word2Vec модель

In [5]:
sentences = [text.split() for text in df['cleaned_website_text']]  # Adjust based on your token format

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

### Создаем векторы для каждого документа

In [ ]:
def document_vector(model, doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in model.wv]
    if not doc:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[doc], axis=0)

X = np.array([document_vector(model, doc) for doc in sentences])
y = df['Category'].values  # Your labels

array([[ 0.31922248, -0.3335363 , -0.158609  , ..., -0.13647515,
        -0.5519677 ,  0.12508087],
       [ 0.40508726, -0.40252197, -0.22987367, ..., -0.18995574,
        -0.4351188 ,  0.19171074],
       [ 0.43943065, -0.51218027, -0.26657584, ..., -0.21445131,
        -0.4331165 ,  0.18033227],
       ...,
       [ 0.46809033,  0.01050375,  0.26246598, ..., -0.38326922,
        -0.831351  ,  0.815428  ],
       [ 0.4431468 , -0.23184232, -0.09211971, ..., -0.0620573 ,
        -0.75520617,  0.40360764],
       [ 0.45871875, -0.417346  , -0.3192415 , ..., -0.09708606,
        -0.6543116 ,  0.363415  ]], dtype=float32)

## Визуализация векторов

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming you have X (word2vec document vectors) and y (labels) from previous steps

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y_encoded)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the neural network
class TextClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
input_dim = X.shape[1]
num_classes = len(le.classes_)
model = TextClassifier(input_dim, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [11]:
# Training loop
num_epochs = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss/len(train_loader.dataset):.4f}')
    print(f'Test Loss: {test_loss/len(test_loader.dataset):.4f}')
    print(f'Accuracy: {100 * correct/total:.2f}%')

Epoch 1/200
Train Loss: 2.6678
Test Loss: 2.6121
Accuracy: 20.43%
Epoch 2/200
Train Loss: 2.5514
Test Loss: 2.3849
Accuracy: 27.24%
Epoch 3/200
Train Loss: 2.3223
Test Loss: 2.0639
Accuracy: 37.99%
Epoch 4/200
Train Loss: 2.0780
Test Loss: 1.8011
Accuracy: 50.90%
Epoch 5/200
Train Loss: 1.8791
Test Loss: 1.6057
Accuracy: 58.06%
Epoch 6/200
Train Loss: 1.7333
Test Loss: 1.5000
Accuracy: 60.93%
Epoch 7/200
Train Loss: 1.6216
Test Loss: 1.3645
Accuracy: 63.44%
Epoch 8/200
Train Loss: 1.5180
Test Loss: 1.3157
Accuracy: 65.59%
Epoch 9/200
Train Loss: 1.4533
Test Loss: 1.2529
Accuracy: 66.67%
Epoch 10/200
Train Loss: 1.4035
Test Loss: 1.2435
Accuracy: 65.95%
Epoch 11/200
Train Loss: 1.3304
Test Loss: 1.2005
Accuracy: 68.10%
Epoch 12/200
Train Loss: 1.2981
Test Loss: 1.1848
Accuracy: 68.82%
Epoch 13/200
Train Loss: 1.2749
Test Loss: 1.1497
Accuracy: 72.04%
Epoch 14/200
Train Loss: 1.2756
Test Loss: 1.1547
Accuracy: 69.89%
Epoch 15/200
Train Loss: 1.1983
Test Loss: 1.1453
Accuracy: 65.95%
Epoc